In [22]:
import argparse
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import optparse
from dotenv import load_dotenv
import pickle

from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds, BrainFlowPresets
from brainflow import ml_model
from brainflow.ml_model import *


In [33]:
def get_data():
    load_dotenv()

    BoardShim.enable_dev_board_logger()

    board_id = 0
    serial_port = os.getenv("SERIAL_PORT")

    params = BrainFlowInputParams()
    params.serial_port = serial_port
    board = BoardShim(board_id, params)
    board.prepare_session()
    sampling_rate = board.get_sampling_rate(board_id)   # per second, presumably

    board.start_stream()
    t = 11
    time.sleep(t)
    # data = board.get_current_board_data (256) # get latest 256 packages or less, doesnt remove them from internal buffer
    data = board.get_board_data()  # get all data and remove it from internal buffer
    board.stop_stream()
    board.release_session()

    return sampling_rate, data

In [138]:
_, data = get_data()

In [139]:

# d2 = data[1:8, -2406:]
d2 = data[1:8, :2406]
dfft =  np.fft.fft(d2).view(np.float64).reshape(7, 2406, 2)
dfft.shape
model = pickle.load(open("finalized_model_l1_C=1e-7.sav", 'rb'))
print(model.predict_proba(dfft.reshape(1, -1)))
model.predict(dfft.reshape(1, -1))

[[0.58540579 0.41459421]]


array([0.])

In [84]:
d2

array([[ 8.99934876e+03,  2.78775427e+03,  3.30135266e+03, ...,
         6.31430075e+03,  4.33500908e+03,  6.19067325e+03],
       [-4.63644470e+03, -1.26381233e+03, -1.35556178e+04, ...,
        -1.92106538e+03, -1.33704559e+04, -2.13394339e+03],
       [ 5.33942942e+03,  4.07939453e+03,  5.44061577e+03, ...,
         8.95681339e+03,  4.66713365e+03,  8.87940930e+03],
       ...,
       [ 7.03474218e+03,  1.94460177e+00,  7.40468590e+03, ...,
         1.01908308e+03,  7.85031263e+03,  8.75294313e+02],
       [ 4.62707932e+03,  2.29076323e+03, -2.57429511e+03, ...,
         5.32112334e+03, -2.63397427e+03,  5.18459889e+03],
       [ 4.29354659e+02,  8.92751025e+02, -2.36233352e+03, ...,
         2.58607448e+03, -3.02007830e+03,  2.44845479e+03]])